In [1]:
import pandas as pd
df_codes = pd.read_csv('failuremode.txt', sep=":", names=["code", "name", "description"])

In [2]:
import pandas as pd
df = pd.read_csv('ai4i2020.csv')

In [3]:
cat_features=['Product ID', 'Type']
X_features= cat_features +  ['Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]']
y_features=[ 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

In [4]:
X=df[X_features]
y=df[y_features]

In [76]:
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.inspection import permutation_importance
import numpy as np
def explain(on, X_train, y_train, cat_features,iterations=300):

    columns = X_train.columns
    _y_train = y_train[on].values.flatten()
    
    clf = CatBoostClassifier(
        iterations=iterations,
        random_seed=42,
        loss_function='Logloss',
        cat_features=cat_features,
        auto_class_weights='Balanced'
    )
    
    clf.fit(
        X_train, _y_train,
        verbose=False
    )


    feature_importance = clf.feature_importances_
    sorted_idx = np.argsort(-1*feature_importance)
    print(on, "importance:", columns[sorted_idx], feature_importance[sorted_idx])

    explain_list = []
    for idx,_y in enumerate(_y_train):

        if _y == 1:
            name = df_codes[df_codes['code']==on]['name'].values[0]
            description = df_codes[df_codes['code']==on]['description'].values[0]

            X_test = X_train.iloc[idx].to_frame().T
            y_test = _y_train[idx]

            #perm_importance = permutation_importance(clf, X_test, np.array([1]), n_repeats=10, random_state=1066)
            #sorted_idx = perm_importance.importances_mean.argsort()
            #col = perm_importance.importances_mean[sorted_idx][0]

            col0 = columns[sorted_idx][0]
            col1 = columns[sorted_idx][1]
            
            explain = f"Found {name} anomaly for Product {X_test['Product ID'].values[0]}: {description} The cause is {col0} which is {X_test[col0].values[0]}. A second cause can be {col1} which is {X_test[col1].values[0]} "
        else:
            explain=""

        explain_list.append(explain)
    
    return np.array(explain_list)

In [77]:
df['anomaly_description'] = ''
for on in y_features:
    explain_list = explain(on, X, y, cat_features)
    df['anomaly_description'] = df['anomaly_description'] + explain_list


TWF importance: Index(['Tool wear [min]', 'Torque [Nm]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Type', 'Air temperature [K]', 'Product ID'],
      dtype='object') [41.30799079 18.54034282 14.51724103 11.43487308  7.65306548  6.5464868
  0.        ]
HDF importance: Index(['Rotational speed [rpm]', 'Air temperature [K]',
       'Process temperature [K]', 'Torque [Nm]', 'Tool wear [min]', 'Type',
       'Product ID'],
      dtype='object') [46.50019171 27.82443679 16.12803496  4.06049659  3.25735017  2.22948979
  0.        ]
PWF importance: Index(['Torque [Nm]', 'Rotational speed [rpm]', 'Type', 'Tool wear [min]',
       'Air temperature [K]', 'Process temperature [K]', 'Product ID'],
      dtype='object') [48.73967738 32.46856129  5.60567914  5.34039737  4.17736495  3.66831986
  0.        ]
OSF importance: Index(['Tool wear [min]', 'Torque [Nm]', 'Type', 'Rotational speed [rpm]',
       'Air temperature [K]', 'Process temperature [K]', 'Product ID'],
      dtype='ob

In [78]:
df[df['anomaly_description']!='']['anomaly_description'].values

array(['Found power failure (PWF) anomaly for Product L47230:  the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset. The cause is Torque [Nm] which is 4.6. A second cause can be Rotational speed [rpm] which is 2861 ',
       'Found power failure (PWF) anomaly for Product L47249:  the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset. The cause is Torque [Nm] which is 65.7. A second cause can be Rotational speed [rpm] which is 1410 Found overstrain failure (OSF) anomaly for Product L47249:  if the product of tool wear and torque exceeds 11,000 minNm for the L product variant (12,000 M, 13,000 H), the process fails due to overstrain. This is true for 98 datapoints. The cause is Tool wear [m

In [80]:
df[df['anomaly_description']!='']['anomaly_description'].to_csv('events.txt', header=False, index=False)